In [1]:
#THIS IS USED FOR TRAINING ON COMPUTER
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
path_test = 'D:/OneDrive - VietNam National University - HCM INTERNATIONAL UNIVERSITY/Code/CapstoneCode/Data/mitbih_test.csv'
path_train = 'D:/OneDrive - VietNam National University - HCM INTERNATIONAL UNIVERSITY/Code/CapstoneCode/Data/mitbih_train.csv'
mit_test = pd.read_csv(path_test,header=None)
mit_train = pd.read_csv(path_train, header=None)

In [2]:
#Replace columns 187 with Class
mit_test.rename(columns={187:"Class"}, inplace=True)
mit_train.rename(columns={187:"Class"}, inplace=True)

In [3]:
# Setting Dictionary to define the type of Heartbeat
# N: normal
#S: Supraventricular premature beat
#P: Premature ventricular contraction
#F: Fusion of ventricular and normal beat
#U: Unclassifiable beat
MIT_Outcome = {0. : 'N',
               1. : 'S',
               2. : 'P',
               3. : 'F',
               4. : 'U'}

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix,  ConfusionMatrixDisplay
from sklearn.utils import resample

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout, InputLayer, LSTM, GRU, BatchNormalization, Bidirectional, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical

In [5]:
#extract class
class_0=mit_train[mit_train["Class"]==0][:10000]
class_1=mit_train[mit_train["Class"]==1]
class_2=mit_train[mit_train["Class"]==2]
class_3=mit_train[mit_train["Class"]==3]
class_4=mit_train[mit_train["Class"]==4]
#upsampling and equalize the number of sample for each class
#class_0_upsample=resample(class_0,replace=True,n_samples=80000,random_state=123)
#class_1_upsample=resample(class_1,replace=True,n_samples=150000,random_state=123)
#class_2_upsample=resample(class_2,replace=True,n_samples=90000,random_state=124)
#class_3_upsample=resample(class_3,replace=True,n_samples=120000,random_state=125)
#class_4_upsample=resample(class_4,replace=True,n_samples=80000,random_state=126)
#class_0_upsample=class_0
#class_1_upsample=resample(class_1,replace=True,n_samples=7000,random_state=123)
#class_2_upsample=resample(class_2,replace=True,n_samples=8000,random_state=124)
#class_3_upsample=resample(class_3,replace=True,n_samples=10000,random_state=125)
#class_4_upsample=resample(class_4,replace=True,n_samples=8000,random_state=126)

#combine
mit_train=pd.concat([class_0,class_1,class_2,class_3,class_4])
print(mit_train.shape)
print(mit_train["Class"].value_counts())

(25083, 188)
0.0    10000
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: Class, dtype: int64


In [6]:
#Split train data into train and valid data
train, valid, out_train, out_valid = train_test_split(mit_train.iloc[:,:187], mit_train["Class"], train_size = 0.8, random_state=42)
test, out_test = mit_test.iloc[:,:187], mit_test["Class"]

#Normalize the data
train = normalize(train, axis = 0, norm='max')
test = normalize(test, axis = 0, norm='max')
valid = normalize(valid, axis = 0, norm='max')

#Change 2D (mxn) data into 3D data (mxnx1)
x_train = train.reshape(train.shape[0],train.shape[1],1)
x_test = test.reshape(test.shape[0],test.shape[1],1)
x_valid = valid.reshape(valid.shape[0],valid.shape[1],1)

#Categorize the target
y_train = to_categorical(out_train)
y_test = to_categorical(out_test)
y_valid = to_categorical(out_valid)

In [7]:
from keras.layers import Dense,Flatten,Conv1D,Activation,Dropout,Input,BatchNormalization, SeparableConv1D
from keras.layers import GlobalMaxPooling1D, Input
from keras import backend as K
from keras import layers
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import MaxPooling1D

In [8]:
def Xception(input_tensor=None,input_shape=None):
    img_input = Input((187,1))
    x = Conv1D(32, 3,strides=2, use_bias=False, name='block1_conv1')(img_input)
    x = BatchNormalization(name='block1_conv1_bn')(x)
    x = Activation('relu', name='block1_conv1_act')(x)
    x = Conv1D(64, 3, use_bias=False, name='block1_conv2')(x)
    x = BatchNormalization(name='block1_conv2_bn')(x)
    x = Activation('relu', name='block1_conv2_act')(x)

    residual = Conv1D(128, 1, strides=2,padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv1D(128, 3, padding='same', use_bias=False, name='block2_sepconv1')(x)
    x = BatchNormalization(name='block2_sepconv1_bn')(x)
    x = Activation('relu', name='block2_sepconv2_act')(x)
    x = SeparableConv1D(128, 3, padding='same', use_bias=False, name='block2_sepconv2')(x)
    x = BatchNormalization(name='block2_sepconv2_bn')(x)

    x = MaxPooling1D(3, strides=2, padding='same', name='block2_pool')(x)
    x = layers.add([x, residual])

    residual = Conv1D(256, 1, strides=2,
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block3_sepconv1_act')(x)
    x = SeparableConv1D(256, 3, padding='same', use_bias=False, name='block3_sepconv1')(x)
    x = BatchNormalization(name='block3_sepconv1_bn')(x)
    x = Activation('relu', name='block3_sepconv2_act')(x)
    x = SeparableConv1D(256, 3, padding='same', use_bias=False, name='block3_sepconv2')(x)
    x = BatchNormalization(name='block3_sepconv2_bn')(x)
    x = MaxPooling1D(3, strides=2, padding='same', name='block3_pool')(x)
    x = layers.add([x, residual])

    residual = Conv1D(728, 1, strides=2, padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block4_sepconv1_act')(x)
    x = SeparableConv1D(728, 3, padding='same', use_bias=False, name='block4_sepconv1')(x)
    x = BatchNormalization(name='block4_sepconv1_bn')(x)
    x = Activation('relu', name='block4_sepconv2_act')(x)
    x = SeparableConv1D(728, 3, padding='same', use_bias=False, name='block4_sepconv2')(x)
    x = BatchNormalization(name='block4_sepconv2_bn')(x)

    x = MaxPooling1D(3, strides=2, padding='same', name='block4_pool')(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = SeparableConv1D(728, 3, padding='same', use_bias=False, name=prefix + '_sepconv1')(x)
        x = BatchNormalization(name=prefix + '_sepconv1_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = SeparableConv1D(728, 3, padding='same', use_bias=False, name=prefix + '_sepconv2')(x)
        x = BatchNormalization(name=prefix + '_sepconv2_bn')(x)
        x = Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = SeparableConv1D(728, 3, padding='same', use_bias=False, name=prefix + '_sepconv3')(x)
        x = BatchNormalization(name=prefix + '_sepconv3_bn')(x)

        x = layers.add([x, residual])

    residual = Conv1D(1024, 1, strides=2,padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu', name='block13_sepconv1_act')(x)
    x = SeparableConv1D(728, 3, padding='same', use_bias=False, name='block13_sepconv1')(x)
    x = BatchNormalization(name='block13_sepconv1_bn')(x)
    x = Activation('relu', name='block13_sepconv2_act')(x)
    x = SeparableConv1D(1024, 3, padding='same', use_bias=False, name='block13_sepconv2')(x)
    x = BatchNormalization(name='block13_sepconv2_bn')(x)

    x = MaxPooling1D(3, strides=2, padding='same', name='block13_pool')(x)
    x = layers.add([x, residual])

    x = SeparableConv1D(1536, 3, padding='same', use_bias=False, name='block14_sepconv1')(x)
    x = BatchNormalization(name='block14_sepconv1_bn')(x)
    x = Activation('relu', name='block14_sepconv1_act')(x)

    x = SeparableConv1D(2048, 3, padding='same', use_bias=False, name='block14_sepconv2')(x)
    x = BatchNormalization(name='block14_sepconv2_bn')(x)
    x = Activation('relu', name='block14_sepconv2_act')(x)
    x = GlobalMaxPooling1D()(x)
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dense(128)(x)
    x = Dense(5, activation='softmax')(x)
    # inputs = img_input
    # Create model.
    # model = Sequential()
    model= Model(img_input,x)
    return model

In [9]:
model_khanh= Xception()
model_khanh.summary()
checkpoint_cb = ModelCheckpoint("CVDs_khanh2.h5", save_best_only=True)
model_khanh.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[tfa.metrics.F1Score(2,"micro")])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 187, 1)]     0           []                               
                                                                                                  
 block1_conv1 (Conv1D)          (None, 93, 32)       96          ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 93, 32)      128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 93, 32)       0           ['block1_conv1_bn[0][0]']    

In [10]:
history_khanh = model_khanh.fit(x_train, y_train, epochs=40, batch_size=32, validation_data=(x_valid, y_valid), callbacks=[checkpoint_cb])

Epoch 1/40
628/628 [==============================] - 65s 74ms/step - loss: 0.4242 - f1_score: 0.8776 - val_loss: 0.8898 - val_f1_score: 0.5814
Epoch 2/40
628/628 [==============================] - 44s 71ms/step - loss: 0.2345 - f1_score: 0.9251 - val_loss: 0.3960 - val_f1_score: 0.8866
Epoch 3/40
628/628 [==============================] - ETA: 0s - loss: 0.9998 - f1_score: 0.8416

KeyboardInterrupt: 

In [ ]:
model_khanh.load_weights("CVDs_khanh2.h5")
#Evaluate train test valid data
eval_train = model_khanh.evaluate(x_train, y_train)
eval_test = model_khanh.evaluate(x_test, y_test)
eval_valid = model_khanh.evaluate(x_valid, y_valid)

In [ ]:
#Predict train, test, valid data
pred_train = model_khanh.predict (x_train)
model_pred_train = np.argmax(pred_train,axis=1)

pred_test = model_khanh.predict (x_test)
model_pred_test = np.argmax(pred_test,axis=1)

pred_valid = model_khanh.predict (x_valid)
model_pred_valid = np.argmax(pred_valid,axis=1)

In [ ]:
print("Predict train outcome")
print(classification_report(out_train, model_pred_train, target_names=[MIT_Outcome[i] for i in MIT_Outcome]))

In [ ]:
print("Predict test outcome")
print(classification_report(out_test, model_pred_test, target_names=[MIT_Outcome[i] for i in MIT_Outcome]))

In [ ]:
print("Predict valid outcome")
print(classification_report(out_valid, model_pred_valid, target_names=[MIT_Outcome[i] for i in MIT_Outcome]))

In [ ]:
cmm = confusion_matrix(y_true= out_test, y_pred=model_pred_test, labels=list(MIT_Outcome.keys()), normalize = 'true')
cm = confusion_matrix(y_true= out_test, y_pred=model_pred_test, labels=list(MIT_Outcome.keys()))
plt.figure(figsize=(8,6))
disp = ConfusionMatrixDisplay(confusion_matrix=cmm,display_labels=[MIT_Outcome[i] for i in MIT_Outcome],)
disp.plot(cmap = "Greys")

TruePositive = np.diag(cm)
FalsePositive = []
for i in range(len(MIT_Outcome)):
    FalsePositive.append(sum(cm[:,i]) - cm[i,i])
FalseNegative = []
for i in range(len(MIT_Outcome)):
    FalseNegative.append(sum(cm[i,:]) - cm[i,i])
TrueNegative = []
for i in range(len(MIT_Outcome)):
    temp = np.delete(cm, i, 0)   # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TrueNegative.append(sum(sum(temp)))
#l = len(y)
#for i in range(len(MIT_Outcome)):
#    print(TruePositive[i] + FalsePositive[i] + FalseNegative[i] + TrueNegative[i] == l)
print("True positive: ", TruePositive[i])
print("False positive: ", FalsePositive[i])
print("False negative: ", FalseNegative[i])
print("True negative: ", TrueNegative[i])

plt.show()

In [ ]:
plt.plot(history_khanh.epoch, history_khanh.history['loss'],
           color='r', label='Train loss')
plt.plot(history_khanh.epoch, history_khanh.history['val_loss'],
           color='b', label='Val loss' , linestyle="--")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import collections
count_predict = collections.Counter(model_pred_test)
model_y_test = np.argmax(y_test, axis = 1)
count_true = collections.Counter(model_y_test)
print(f"Predict = {count_predict}")
print(f"True = {count_true}")